1. Ознакомьтесь с данными , приведенными в таблице 1.1, их графическим изображением на диаграмме 1 и выводом, сделанным на основе графического анализа.
2. Самостоятельно сформируйте новые ряды исходных данных (показатели x, y, z) и занесите их в таблицу 1.2. Показатель x будем считать результирующим, а показатели y и z - факторными (причинными). По мере занесения данных в таблицу 2, на диаграмме 2 будет появляться их графическое изображение. Сделайте вывод на основе графического анализа по образцу.
3. Рассчитайте (пользуясь таблицей 2.1. на листе 2) линейные коэффициенты корреляции между показателями x и y, а также между показателями x и z. Сделайте выводы о том, какая связь более сильная. Тот показатель (y или z), у которого связь с показателем x более сильная, выберите для дальнейших расчетов.
4. Сделайте прогноз показателя x по двум вариантам уравнений тренда (линейному и квадратичному). Для расчета параметров уравнений тренда пользуйтесь вспомогательной таблицей на листе 3. Рассчитайте ошибки аппроксимации и сделайте вывод о том, какой прогноз более достоверен.
5. Рассчитайте параметры уравнения парной линейной  регрессии, выражающей зависимость между показателем x и тем из двух показателей (y или z), с которым связь показателя x более сильная. Пользуйтесь вспомогательными таблицами на листах 4 и 5. Рассчитайте ошибку аппроксимации и индекс детерминации. Сделайте вывод о том, насколько хорошо построенное уравнение отражает существующую зависимость.
6. Выполните (на листе 6) прогноз показателя, выбранного в п.3, по любому из уравнений тренда. Затем подставьте полученное прогнозное значение в уравнение регрессии между этим показателем и показателем x и рассчитайте прогноз спроса по уравнению регрессии.

In [ ]:
import pandas as pd
import plotly.express as px
import seaborn as sns
from matplotlib import pyplot as plt

In [ ]:
df = pd.read_csv("../data/chapter_1.csv", delimiter=";")
df.head(5)

In [ ]:
df.info()

In [ ]:
df.describe()

# Task 3
---
Рассчитайте (пользуясь таблицей 2.1. на листе 2) линейные коэффициенты корреляции между показателями x и y,
а также между показателями x и z. Сделайте выводы о том, какая связь более сильная.
Тот показатель (y или z), у которого связь с показателем x более сильная, выберите для дальнейших расчетов.

In [ ]:
plt.title("Linear correlation")
sns.heatmap(df.corr(), annot=True, fmt=".2f", cmap="coolwarm")

In [ ]:
x_demand_y_produced = px.scatter(
    df,
    x=df.x_demand,
    y=df.y_produced,
    trendline="lowess",
    trendline_options=dict(frac=0.1),
)
x_demand_y_produced.show()
x_demand_z_price = px.scatter(
    df,
    x=df.x_demand,
    y=df.z_price,
    trendline="lowess",
    trendline_options=dict(frac=0.1),
)
x_demand_z_price.show()

# Task 4
---
Сделайте прогноз показателя x по двум вариантам уравнений тренда (линейному и квадратичному).
Для расчета параметров уравнений тренда пользуйтесь вспомогательной таблицей на листе 3.
Рассчитайте ошибки аппроксимации и сделайте вывод о том, какой прогноз более достоверен.

### Linear trend

In [ ]:
import numpy as np

period = df["period"].to_numpy()
x_demand = df["x_demand"].to_numpy()
y_produced = df["y_produced"].to_numpy()
z_price = df["z_price"].to_numpy()

future_period = np.array([[x] for x in range(2022, 2032)])

In [ ]:
from pathlib import Path

Path("../images").mkdir(exist_ok=True)

In [ ]:
import plotly.graph_objects as go
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

(
    period_train,
    period_test,
    x_demand_train,
    x_demand_test,
    y_produced_train,
    y_produced_test,
    z_price_train,
    z_price_test,
) = train_test_split(
    period, x_demand, y_produced, z_price, test_size=0.2, random_state=42
)

linear_reg_demand = LinearRegression()
linear_reg_demand.fit(period_train.reshape(-1, 1), x_demand_train.reshape(-1, 1))
x_demand_prediction = linear_reg_demand.predict(future_period)

linear_reg_produced = LinearRegression()
linear_reg_produced.fit(period_train.reshape(-1, 1), y_produced_train.reshape(-1, 1))
y_produced_prediction = linear_reg_produced.predict(future_period)

linear_reg_price = LinearRegression()
linear_reg_price.fit(period_train.reshape(-1, 1), z_price_train.reshape(-1, 1))
z_price_prediction = linear_reg_price.predict(future_period)

x_demand_trace = go.Scatter(
    x=period, y=x_demand, mode="markers", name="Actual Data: x_demand"
)
y_produced_trace = go.Scatter(
    x=period, y=y_produced, mode="markers", name="Actual Data: y_produced"
)
z_price_trace = go.Scatter(
    x=period, y=z_price, mode="markers", name="Actual Data: z_price"
)

x_demand_reg_trace = go.Scatter(
    x=period_train,
    y=linear_reg_demand.predict(period_train.reshape(-1, 1)).flatten(),
    mode="lines",
    name="^ Regression: x_demand",
)
y_produced_reg_trace = go.Scatter(
    x=period_train,
    y=linear_reg_produced.predict(period_train.reshape(-1, 1)).flatten(),
    mode="lines",
    name="^ Regression: y_produced",
)
z_price_reg_trace = go.Scatter(
    x=period_train,
    y=linear_reg_price.predict(period_train.reshape(-1, 1)).flatten(),
    mode="lines",
    name="^ Regression: z_price",
)

x_demand_pred_trace = go.Scatter(
    x=future_period.flatten(),
    y=x_demand_prediction.flatten(),
    mode="markers",
    name="^ Prediction: x_demand",
)
y_produced_pred_trace = go.Scatter(
    x=future_period.flatten(),
    y=y_produced_prediction.flatten(),
    mode="markers",
    name="^ Prediction: y_produced",
)
z_price_pred_trace = go.Scatter(
    x=future_period.flatten(),
    y=z_price_prediction.flatten(),
    mode="markers",
    name="^ Prediction: z_price",
)

layout = go.Layout(
    title="^ Regression Predictions",
    xaxis=dict(title="Period"),
    yaxis=dict(title="Value"),
)

fig = go.Figure(
    data=[
        x_demand_trace,
        y_produced_trace,
        z_price_trace,
        x_demand_reg_trace,
        y_produced_reg_trace,
        z_price_reg_trace,
        x_demand_pred_trace,
        y_produced_pred_trace,
        z_price_pred_trace,
    ],
    layout=layout,
)
fig.show()

In [ ]:
print("x_demand_prediction\n", x_demand_prediction)
print("z_price_prediction\n", z_price_prediction)

In [ ]:
from sklearn.metrics import mean_absolute_error

print(
    "MAE x_demand: "
    + str(mean_absolute_error(x_demand.reshape(-1, 1), x_demand_prediction))
)
print(
    "MAE z_price: "
    + str(mean_absolute_error(z_price.reshape(-1, 1), z_price_prediction))
)

### Quadratic trend

In [ ]:
import plotly.graph_objects as go
from sklearn.preprocessing import PolynomialFeatures

polynomial_features = PolynomialFeatures(degree=2)


def quadratic_regression(future_period, target_variable):
    period_reshaped = period.reshape(-1, 1)
    target_variable_reshaped = target_variable.reshape(-1, 1)

    period_poly = polynomial_features.fit_transform(period_reshaped)

    quadratic_reg = LinearRegression()
    quadratic_reg.fit(period_poly, target_variable_reshaped)

    future_period_poly = polynomial_features.transform(future_period)
    target_prediction = quadratic_reg.predict(future_period_poly)

    return quadratic_reg, target_prediction


quadratic_reg_x_demand, x_demand_prediction = quadratic_regression(
    future_period, x_demand
)
quadratic_reg_y_produced, y_produced_prediction = quadratic_regression(
    future_period, y_produced
)
quadratic_reg_z_price, z_price_prediction = quadratic_regression(future_period, z_price)

x_demand_trace = go.Scatter(
    x=period, y=x_demand, mode="markers", name="Actual Data: x_demand"
)
y_produced_trace = go.Scatter(
    x=period, y=y_produced, mode="markers", name="Actual Data: y_produced"
)
z_price_trace = go.Scatter(
    x=period, y=z_price, mode="markers", name="Actual Data: z_price"
)

x_demand_reg_trace = go.Scatter(
    x=period,
    y=quadratic_reg_x_demand.predict(
        polynomial_features.transform(period.reshape(-1, 1))
    ),
    mode="lines",
    name="^^ Regression: x_demand",
)
y_produced_reg_trace = go.Scatter(
    x=period,
    y=quadratic_reg_y_produced.predict(
        polynomial_features.transform(period.reshape(-1, 1))
    ),
    mode="lines",
    name="^^ Regression: y_produced",
)
z_price_reg_trace = go.Scatter(
    x=period,
    y=quadratic_reg_z_price.predict(
        polynomial_features.transform(period.reshape(-1, 1))
    ),
    mode="lines",
    name="^^ Regression: z_price",
)

x_demand_pred_trace = go.Scatter(
    x=future_period.flatten(),
    y=x_demand_prediction.flatten(),
    mode="markers",
    name="^^ Prediction: x_demand",
)
y_produced_pred_trace = go.Scatter(
    x=future_period.flatten(),
    y=y_produced_prediction.flatten(),
    mode="markers",
    name="^^ Prediction: y_produced",
)
z_price_pred_trace = go.Scatter(
    x=future_period.flatten(),
    y=z_price_prediction.flatten(),
    mode="markers",
    name="^^ Prediction: z_price",
)

layout = go.Layout(
    title="^^ Regression Predictions",
    xaxis=dict(title="Period"),
    yaxis=dict(title="Value"),
)

fig = go.Figure(
    data=[
        x_demand_trace,
        y_produced_trace,
        z_price_trace,
        x_demand_reg_trace,
        y_produced_reg_trace,
        z_price_reg_trace,
        x_demand_pred_trace,
        y_produced_pred_trace,
        z_price_pred_trace,
    ]
)
fig.update_layout(title="^^ Regression", xaxis_title="Year", yaxis_title="Value")

fig.show()

In [ ]:
print("x_demand_prediction\n", x_demand_prediction)
print("z_price_prediction\n", z_price_prediction)

In [ ]:
from sklearn.metrics import mean_absolute_error

print("MAE: " + str(mean_absolute_error(x_demand.reshape(-1, 1), x_demand_prediction)))

### Linear regression vs Quadratic regression

In [ ]:
import numpy as np
import plotly.graph_objects as go
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression

period_reshaped = np.array([[x] for x in range(2022, 2030)])

# Linear regression
linear_reg = LinearRegression()
linear_reg.fit(period.reshape(-1, 1), x_demand)
linear_x_demand_prediction = linear_reg.predict(period_reshaped)

# Quadratic regression
quadratic_features = PolynomialFeatures(degree=2)
period_poly = quadratic_features.fit_transform(period.reshape(-1, 1))
quadratic_reg = LinearRegression()
quadratic_reg.fit(period_poly, x_demand)
quadratic_x_demand_prediction = quadratic_reg.predict(
    quadratic_features.transform(period_reshaped)
)

# Create plotly traces
linear_trace = go.Scatter(x=period, y=x_demand, mode="markers", name="Actual Data")

linear_prediction_trace = go.Scatter(
    x=period_reshaped.flatten(),
    y=linear_x_demand_prediction,
    mode="lines",
    name="Linear Regression Prediction",
)

quadratic_trace = go.Scatter(x=period, y=x_demand, mode="markers", name="Actual Data")

quadratic_prediction_trace = go.Scatter(
    x=period_reshaped.flatten(),
    y=quadratic_x_demand_prediction,
    mode="lines",
    name="Quadratic Regression Prediction",
)

# Create the figure
fig = go.Figure(
    data=[
        linear_trace,
        linear_prediction_trace,
        quadratic_trace,
        quadratic_prediction_trace,
    ]
)

# Add layout information
fig.update_layout(
    title="Linear vs Quadratic Regression", xaxis_title="Year", yaxis_title="x_demand"
)

# Show the plot
fig.show()